In [4]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 6610
# Algorithms


## Fast Fourier Transform


So far we have discussed divide-and-conquer algorithms for:

- Integer Multiplication
- Matrix Multiplication

Can we multiply polynomials quickly? Why would we want to? 


## Convolutions, Signal Processing and Polynomial Multiplication

Recall that the dot product of two vectors $\mathbf{a}=\langle a_0, \ldots, a_{n-1}\rangle$ and $\mathbf{b}=\langle b_0, \ldots, b_{n-1}\rangle$ is given by:

$$\sum_{i=0}^{n-1} a_i \cdot b_i$$

The *convolution* of $\mathbf{a}*\mathbf{b}$ has:

$$(\mathbf{a}*\mathbf{b})_k = \sum_{i+j=k} a_i\cdot b_j$$


What is $\langle 1, 2, 3\rangle * \langle 4, 5, 6 \rangle$? How many entries can a convolution of two vectors of length $n$ have?




The convolution of $\langle 1, 2, 3\rangle * \langle 4, 5, 6 \rangle$ is $\langle 4, 13, 28, 27, 18\rangle$. The result of convoluting two vectors of length $n$ is $2n-1$. 

What is the work/span of computing the convolution?

Why are convolutions important/interesting?

* **Signal processing**: If we let $\mathbf{a}$ be a discrete signal, and $\mathbf{b}$ be a "mask", then the convolution gives us a smoothed signal!

* **Polynomials**: If $\mathbf{a}$ and $\mathbf{b}$ represent polynomial coefficients, then the convolution gives us the product polynomial!


## Polynomial Multiplication

Let $\mathbf{a}$ represent $a(x) = \sum_{i=0}^{n-1} a_i\cdot x^i$ and let $\mathbf{b}$ represent $b(x) = \sum_{i=0}^{n-1} b_i\cdot x^i$. Then the product $c(x) = a(x)\cdot b(x)$ has degree $2n-2$ and the coefficient of $x^k$ in $c(x)$ is the sum of the product of coefficients $x^i$ and $x^j$ where $x^k = x^i\cdot x^j$. This is exactly the entry $(\mathbf{a}*\mathbf{b})_k$!

So, we can multiply polynomials in $O(n^2)$ work and span. But can we do better?



Is the coefficient representation the best for a polynomial? We know that a polynomial of degree $d$ can be uniquely represented by $d+1$ points. So why not represent the polynomials in *point-value* form on the same points? Then we can multiply values in $O(n)$ work/span to get the the product.

So the algorithm for polynomial multiplication would then have the following steps:

* **Evaluate**: Evaluate $a(x)$ and $b(x)$ on points $x_0, x_1, \ldots x_{2n-1}$.
* **Multiply**: Compute $c(x_j) =  a(x_j)\cdot b(x_j)$ for $j = 0, 1, \ldots 2n-1$.
* **Interpolate**: Recover coefficients $c_0, \ldots c_{2n-1}$ from values $c(x_j)$.

What is the running time of these steps?

Moreover we saw that the coefficients are what we really want -- so we still need to perform **interpolation** on the values to recover the coefficients.


Actually, the Fundamental Theorem of Algebra states that every polynomial of degree $d$ has $d$ **complex** roots.

We will use the roots of a special polynomial, called the *roots of unity*.

An $n^{th}$ root of unity $\omega_{j, n}$ is defined as $e^{2\pi i \cdot j/n}$ for $j=0,\ldots,n-1$ where $i=\sqrt{-1}$. $\omega_{j,n}$ are called a *roots of unity* because they are solutions to $x^n - 1 = 0.$

Note that $\omega^2_{j, n} = \omega_{j, n/2}$. Why?


Back to divide and conquer - how do we split/divide polynomials?

We can simply use this fact:

$$a(x) = a_{\mathsf{\small even}}(x^2) + x \cdot a_{\mathsf{\small odd}}(x^2)$$

where $a_{\mathsf{\small even}}$ contains the even coefficients of $a(x)$ and $a_{\mathsf{\small odd}}(x)$ contains the odd coefficients.



What happens when we evaluate $a(x)$ on $n^{th}$ roots of unity?

$$ a(\omega_{j, n}) = a_{\mathsf{\small even}}(\omega^2_{j, n}) + \omega_{j, n} \cdot a_{\mathsf{\small odd}}(\omega^2_{j, n}) $$

Or, using our basic lemma about squaring $n^{th}$ roots of unity:

$$ a(\omega_{j, n}) = a_{\mathsf{\small even}}(\omega_{j, n/2}) + \omega_{j, n} \cdot a_{\mathsf{\small odd}}(\omega_{j, n/2}) $$

This means that evaluating a polynomial of degree $n$ on $n^{th}$ roots of unity can be done by evaluating **two** polynomials of degree $n/2$ on $(n/2)^{th}$ roots of unity! Once we recursively get the values of $a$ on the $(n/2)^{th}$ roots of unity, we can combine results to obtain the values of $a$ on the $n^{th}$ roots of unity.


Thus the total work would be:

$$ W(n) = 2W(n/2) + cn = O(n \log n).$$

and the span is:

$$ S(n) = S(n/2) + c = O(\log n).$$



## Polynomial Interpolation

To review our steps so far:

* **Evaluate**: Evaluate $a(x)$ and $b(x)$ on points $\omega_{j, 2n}$ for $j=0, 1, \ldots, 2n-1$.
* **Multiply**: Compute $c(\omega_{j, 2n}) =  a(\omega_{j, 2n})\cdot b(\omega_{j, 2n})$ for $j = 0, 1, \ldots 2n-1$.
* **Interpolate**: Recover coefficients $c_0, \ldots c_{2n-1}$ from values $c(\omega_{j, 2n})$.

We've shown how to quickly multiply two polynomials in point-value representation over roots of unity. But, how do we get the coefficients of the product polynomial?




Let 

$$d(x) = \sum_{j=0}^{2n-1} = c(\omega_{j,2n})\cdot x^j $$

What is the cost to evaluate $d(x)$ on the $2n^{th}$ roots of unity?

It would take $O(n\log n)$ work - but so what?

Let us take a closer look at the evaluation of polynomial $a(x)$ at the $n^{th}$ roots of unity. For $0\leq j < n$:
    
    
$$a(\omega_{j,n}) =  \sum_{k=0}^{n-1} a_k \omega^k_{j,n}$$

We can write this in matrix form as $\mathbf{a_\omega} = V\cdot\mathbf{a}$. Now $V$ is a *Vandermonde* matrix, which has the property that it is invertible (since the entries are powers of the distinct $n^{th}$ roots of unity).

This means that if we compute $V^{-1}\cdot\mathbf{a_\omega} = V^{-1}\cdot V\cdot\mathbf{a}$, we immediately have the coefficients we seek. But this requires a matrix-vector multiplication! Can we do better?



It turns out that this inversion is performed if we evaluate $d(x)$ at the $2n^{th}$ roots of unity! That is:

$$ d(\omega_{j, 2n}) = 2n \cdot c_j $$

This is because, at position $V^{-1}_{(j,k)} = \omega^{-kj}_n/n$. This means that when we multiply a row $j$ of $V^{-1}$ with a column $j'$ of $V$, we get:

$$(V^{-1}V)_{j,j'} = \sum_{k=0}^n (\omega^{-kj}_n/n)(\omega^{kj'}_n) = \sum_{k=0}^n \omega^{k(j'-j)}_n/n $$

Now, it turns out that this summation is 0 when $j=j'$ and 0 otherwise. To make this observation we need the following:


**Lemma**:  For any $n\geq 1$ and nonnegative integer $k$ that is not divisible by $n$:  

$$\sum_{j=0}^{n-1} (\omega^k_n)^j  = 0.$$

Can you prove it?


To sum up all of our steps for computing $\mathbf{a}*\mathbf{b}$:

* **Evaluate**: Evaluate $a(x)$ and $b(x)$ on points $\omega_{j, 2n-1}$ for $j=0, 1, \ldots, 2n-1$.
* **Multiply**: Compute $c(\omega_{j,2n}) =  a(\omega_{j,2n})\cdot b(\omega_{j,2n})$ for $j = 0, 1, \ldots 2n-1$.
* **Interpolate**: Let $d(x) = \sum_{j=0}^{2n-1} = c(\omega_{j,2n})\cdot x^j$. Recover the coefficients $c_0, \ldots c_{2n-1}$ by evaluating $d(x_j)$ on the $2n^{th}$ roots of unity.

The evaluation steps take $O(n \log n)$ work and $O(\log n)$ span, while the multiplication step takes $O(n)$ work and $O(1)$ span. Thus the work to compute $\mathbf{a}*\mathbf{b}$ is $O(n\log n)$ and the span is $O(\log n)$.
